<center>

**PRUEBAS DE CERO CONOCIMIENTO**

</center>

<p align="center">
    <img src="https://logowik.com/content/uploads/images/escudo-de-la-universidad-nacional-de-colombia-20163327.logowik.com.webp" width="400">
</p>

<center>

# **🧾PRUEBAS DE CERO CONOCIMIENTO📜**

<p align="center">
    <img src="https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEgIDVV-taI6gDRGwUWAJve90Hkn6beUw0LzaSMK8x2KAQmctESnTDhE5hBYcPlN93f0LWlVtSuz-Lsq_te8XC0mutR3oJvantMeWH3Sdc-mTrJdQGdgK-yPSWLlrJL89W7oxX06-vJox3c/s1600/grafos+isomfs.jpg" width="400">
</p>

<div align="justify">

**Problema de Isomorfismo de Grafos**

El **problema de isomorfismo de grafos** se formula así:

> Dados dos grafos simples $G_{0} = (V, E_{0}),\quad G_{1} = (V, E_{1}),$ con el mismo conjunto de vértices \(V\) de tamaño \(n\), decidir si existe una permutación  $\pi \in S_{n}$ tal que $G_{1} \;=\; \pi(G_{0})$,
 es decir, si podemos relabelar los vértices de $G_{0}$ por $\pi$ para obtener exactamente $G_{1}$.

---

**Notación Matemática**

- $V = \{0,1,\dots,n-1\}$.  
- Una permutación $\pi\in S_{n}$ actúa sobre aristas:  
  $$
   (u,v)\in E_{0}
   \quad\Longleftrightarrow\quad
   (\pi(u),\,\pi(v))\in E_{1}.
  $$
- Denotamos $\pi(G_{0})$ al grafo cuyos vértices están relabelados por $\pi$.

---

**Complejidad**

- Se sitúa en la clase **GI** (Graph Isomorphism).  
- **NI**: no se conoce si GI es NP-completo ni si está en P, aunque recientes avances lo sitúan en **quasi-polinomial**.  
- Importante en teoría de la complejidad y en criptografía post-cuántica.

---

**Enfoque con Zero-Knowledge Proof**

En el notebook que sigue implementamos:

1. **Commit**: Prover “mezcla” $G_{0}$ con una permutación aleatoria $\sigma$ y publica $H = \sigma(G_{0})$.  
2. **Challenge**: Verifier elige un bit $b\in\{0,1\}$.  
3. **Response**:  
   - Si $b=0$, Prover revela $\sigma$.  
   - Si $b=1$, revela $\sigma \circ \pi^{-1}$.  
4. **Verify**: Verifier comprueba que la permutación revelada transforma $G_{b}$ en $H$.

Este protocolo demuestra conocimiento de la permutación secreta $\pi$ sin revelarla, con error de soundness \(1/2\) por iteración y completa zero-knowledge mediante simulación.  


</div>

**📥Instalación paquetes📦**

In [5]:

!pip install -q networkx matplotlib ipywidgets
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json


**📥Importaciones📦**

In [6]:


import networkx as nx
import matplotlib.pyplot as plt
import random
from ipywidgets import (
    Textarea, BoundedIntText, Button, Output,
    GridspecLayout, VBox, HBox, Layout, Label, HTML
)
from IPython.display import display, clear_output


**👨‍💻Implementación👩‍💻**

In [7]:

plt.rcParams.update({
    'figure.figsize': (3,3),
    'axes.titlesize': 'medium'
})

**👨‍💻Implementación👩‍💻**

In [8]:

class GraphZKDemo:
    def __init__(self, default_n=6):
        self.n = default_n
        self._build_ui()
        display(self.container)

    def _build_ui(self):

        self.input_n = BoundedIntText(value=self.n, min=2, max=20,
            description='Nodos (n):', layout=Layout(width='200px'))
        self.input_edges = Textarea(
            value='0-1,0-2,1-2', description='Aristas G₀:',
            layout=Layout(width='400px', height='80px')
        )
        self.input_pi = Textarea(
            value=','.join(str(i) for i in range(self.n)),
            description='Perm π:', layout=Layout(width='400px', height='50px')
        )
        self.btn_load = Button(description='1. Cargar G₀ y π', button_style='info')
        self.out_load = Output(layout=Layout(border='1px solid #ccc', padding='5px'))
        self.btn_load.on_click(self._on_load)


        self.input_sigma = Textarea(
            placeholder='Opcional: 3,0,1,...', description='σ:',
            layout=Layout(width='400px', height='50px')
        )
        self.btn_commit = Button(description='2. Commit σ', button_style='warning', disabled=True)
        self.out_commit = Output(layout=Layout(border='1px solid #ccc', padding='5px'))
        self.btn_commit.on_click(self._on_commit)

        self.btn_b0 = Button(description='3. Desafío b=0', button_style='primary', disabled=True)
        self.btn_b1 = Button(description='3. Desafío b=1', button_style='primary', disabled=True)
        self.out_challenge = Output(layout=Layout(border='1px solid #ccc', padding='5px'))
        self.btn_b0.on_click(lambda _: self._on_challenge(0))
        self.btn_b1.on_click(lambda _: self._on_challenge(1))

        self.input_resp = Textarea(description='Resp:', disabled=True,
            layout=Layout(width='400px', height='50px'), placeholder='Permutación revelada')
        self.btn_reveal = Button(description='4. Reveal', button_style='info', disabled=True)
        self.out_reveal = Output(layout=Layout(border='1px solid #ccc', padding='5px'))
        self.btn_reveal.on_click(self._on_reveal)

        self.btn_verify = Button(description='5. Verify', button_style='success', disabled=True)
        self.out_verify = Output(layout=Layout(border='1px solid #ccc', padding='5px'))
        self.btn_verify.on_click(self._on_verify)


        self.gs = GridspecLayout(1, 3, width='100%')
        self.gs[0,0] = VBox([Label('G₀'), Output()], layout=Layout(align_items='center'))
        self.gs[0,1] = VBox([Label('H'),  Output()], layout=Layout(align_items='center'))
        self.gs[0,2] = VBox([Label('G_b'), Output()], layout=Layout(align_items='center'))

        # Contenedor principal
        controls = VBox([
            HBox([self.input_n, self.btn_load]),
            self.input_edges,
            self.input_pi,
            self.out_load,
            HBox([self.input_sigma, self.btn_commit]),
            self.out_commit,
            HBox([self.btn_b0, self.btn_b1]),
            self.out_challenge,
            HBox([self.btn_reveal, self.input_resp, self.btn_verify]),
            self.out_reveal,
            self.out_verify
        ], layout=Layout(width='100%'))

        self.container = VBox([
            HTML('<h3>ZK Isomorfismo de Grafos</h3>'),
            controls,
            self.gs
        ])

    def _draw(self, G, out_widget, title):
        out_widget.clear_output()
        with out_widget:
            plt.clf()
            pos = nx.circular_layout(G)
            nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray')
            plt.title(title)
            plt.show()

    def _parse_edges(self, text):
        return [(int(u), int(v)) for token in text.split(',') if '-' in token for u,v in [token.strip().split('-')]]

    def _parse_perm(self, text, n):
        lst = [int(x) for x in text.split(',') if x.strip()!='']
        if len(lst)!=n:
            raise ValueError(f'Perm debe tener {n} elementos')
        return lst

    def _on_load(self, _):
        self.out_load.clear_output()
        try:
            n = self.input_n.value
            edges = self._parse_edges(self.input_edges.value)
            pi = self._parse_perm(self.input_pi.value, n)
            G0 = nx.Graph(); G0.add_nodes_from(range(n)); G0.add_edges_from(edges)
            G1 = nx.relabel_nodes(G0, {i:pi[i] for i in range(n)})
            self.G0, self.G1, self.pi, self.n = G0, G1, pi, n
            with self.out_load:
                print('✅ Cargado: G₀ y π')
            self._draw(self.G0, self.gs[0,0].children[1], 'G₀')
            self._draw(self.G1, self.gs[0,2].children[1], 'G₁ = π(G₀)')
            self.btn_commit.disabled = False
        except Exception as e:
            with self.out_load:
                print('❌ Error:', e)

    def _on_commit(self, _):
        self.out_commit.clear_output()
        try:
            sigma = self._parse_perm(self.input_sigma.value, self.n) if self.input_sigma.value else random.sample(range(self.n), self.n)
            H = nx.relabel_nodes(self.G0, {i:sigma[i] for i in range(self.n)})
            self.sigma, self.H = sigma, H
            with self.out_commit:
                print(f'🔐 Commit: σ = {sigma}')
            self._draw(self.H, self.gs[0,1].children[1], 'H = σ(G₀)')
            self.btn_b0.disabled = self.btn_b1.disabled = False
            self.btn_commit.disabled = True
        except Exception as e:
            with self.out_commit:
                print('❌ σ inválida:', e)

    def _on_challenge(self, b):
        self.out_challenge.clear_output()
        with self.out_challenge:
            print(f'❓ Desafío b = {b}')
        self.b = b
        self.btn_reveal.disabled = False
        self.btn_b0.disabled = self.btn_b1.disabled = True

    def _on_reveal(self, _):
        self.out_reveal.clear_output()
        pi_inv = {self.pi[i]:i for i in range(self.n)}
        if self.b==0:
            resp, Gb, desc = self.sigma, self.G0, 'σ'
        else:
            resp = [self.sigma[pi_inv[i]] for i in range(self.n)]
            Gb, desc = self.G1, 'σ ∘ π⁻¹'
        self.resp, self.Gb = resp, Gb
        self.input_resp.value = ','.join(map(str,resp))
        self.input_resp.disabled = False
        with self.out_reveal:
            print(f'📤 Reveal ({desc}): {resp}')
        self._draw(self.Gb, self.gs[0,2].children[1], f'G_b (b={self.b})')
        self.btn_reveal.disabled = True
        self.btn_verify.disabled = False

    def _on_verify(self, _):
        self.out_verify.clear_output()
        try:

            user_resp = self._parse_perm(self.input_resp.value, self.n)
            # Aplica la permutación a G_b
            mapping = {i: user_resp[i] for i in range(self.n)}
            H_ver = nx.relabel_nodes(self.Gb, mapping)


            ok = (set(H_ver.nodes()) == set(self.H.nodes())
                  and set(H_ver.edges()) == set(self.H.edges()))

            with self.out_verify:
                print('✔️ Verificación:', '✅ Correcto' if ok else '❌ Incorrecto')
        except Exception as e:
            with self.out_verify:
                print('❌ Error en resp:', e)
        self.btn_verify.disabled = True

demo = GraphZKDemo(default_n=6)
